In [ ]:
#default_exp prodigy_demo

In [ ]:
#no_test

#data structure imports
import pandas as pd
import numpy as np

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *

In [ ]:
#no_test
seed = 2436
random.seed(seed)
fix_random_seed(seed)

# First Book of Baptisms, Matanzas (15834)

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/matanzas_baut_1_rel_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
13,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Paula,10,15,PER
14,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Esc.a,17,22,CHAR
15,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Juebes veinte y tres de feb.o de mil sietec.to...,23,86,DATE
16,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Thomas de Orvera,90,106,PER
17,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Paula,145,150,PER
18,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,h. l.16,151,158,CHAR
19,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Juan Joseph,162,173,PER
20,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Maria Josepha,177,190,PER
21,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,esc.s,191,196,CHAR
22,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Capitan D. Luis Hurtado de Mendoza,201,235,PER


In [ ]:
#no_test

train_spacy[:2]

[('[margin]: Paula. Esc.a Juebes veinte y tres de feb.o de mil sietec.tos. y diez y nueve Yo Thomas de Orvera baptizé, y pusse los santos15 oleos á Paula h. l.16 de Juan Joseph, y Maria Josepha esc.s del Capitan D. Luis Hurtado de Mendoza fue su Padrino Bartholome Rixo, y lo firmé. [signed]: Thomas de Orvera ',
  {'entities': [(10, 15, 'PER'),
    (17, 22, 'CHAR'),
    (23, 86, 'DATE'),
    (90, 106, 'PER'),
    (145, 150, 'PER'),
    (151, 158, 'CHAR'),
    (162, 173, 'PER'),
    (177, 190, 'PER'),
    (191, 196, 'CHAR'),
    (201, 235, 'PER'),
    (243, 250, 'REL'),
    (251, 266, 'PER'),
    (290, 306, 'PER')]}),
 ('[margin]: Bernardo Esc.vo Domingo nueve de Abril de mil sietectos y diez y nueve Yo Thomas de Orvera baptize, y pusse los s.tos oleos á Bernardo negro adulto de nacion carabali esc.o de D. Juan Joseph de Justis fue su P. Andres de Morales, y lo firmé. [signed]: Thomas de Orvera ',
  {'entities': [(10, 18, 'PER'),
    (19, 25, 'CHAR'),
    (26, 80, 'DATE'),
    (84, 100, 

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/mat_baut_1'

#load model from pretrained
nlp_model = load_model('es_core_news_md')

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,8065.185068,86.27451,9.977324,17.886179


Saved model to models\mat_baut_1


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,6746.61611,100.0,5.21542,9.913793


Performance hasn't improved for 1 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,6373.28776,45.762712,6.122449,10.8


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,44.0,5977.159367,83.412322,79.818594,81.575898


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,45.0,5952.184566,84.745763,79.365079,81.967213


Performance hasn't improved for 5 cycles...
Done training after 45 meta cycles.


# Second Book of Baptisms, St. Augustine (239746)

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/st_aug_bapt_1793_1807_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
31,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Antonio Guillo,3,17,PER
32,2,"2. Antonio Guillo Miercoles, día veinte de Nov...","Miercoles, día veinte de Noviembre de Mil Sete...",18,86,DATE
33,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Don Miguel o’Reilly,90,109,PER
34,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Teniente de Cura Beneficiado,110,138,CHAR
35,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Vicario,140,147,CHAR
36,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Juez Eclesiás tico Auxiliar,151,178,CHAR
37,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Iglesia parroquial,187,205,LOC
38,2,"2. Antonio Guillo Miercoles, día veinte de Nov...","provincia, de San Agustín",208,233,LOC
39,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Florida Oriental,240,256,LOC
40,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",niño,295,299,CHAR


In [ ]:
#no_test

train_spacy[:2]

[('2. Antonio Guillo Miercoles, día veinte de Noviembre de Mil Setecientos Noventa y Tres Yo Don Miguel o’Reilly Teniente de Cura Beneficiado, Vicario, y Juez Eclesiás tico Auxiliar de esta Iglesia parroquial y provincia, de San Agustín de la Florida Oriental. Baptisé y puse los santos oleos a un niño de color, moreno, que nacio el día primero de octubre del presente año, hijo na tural de los negros libres Isaac Guillo y Sara Ca mel, ambos naturales de America del Norte; y y en el exerci las sacras Ceremonias, y preces de nuestra Santa madre Iglesia, y la puse por nombre Juan Antonio. Fueron sus padrinos Antonio Pellice, y Susana Pellicer vecinos de esta Ciudad, les advertí el parentesco espiritual, y demás obligaciones, y lo firmé en dicho dia, mes y año.  Miguel OReilly ',
  {'entities': [(3, 17, 'PER'),
    (18, 86, 'DATE'),
    (90, 109, 'PER'),
    (110, 138, 'CHAR'),
    (140, 147, 'CHAR'),
    (151, 178, 'CHAR'),
    (187, 205, 'LOC'),
    (208, 233, 'LOC'),
    (240, 256, 'LOC'

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/st_aug_bapt_2'

#load model from pretrained
nlp_model = load_model('es_core_news_md')

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,10314.124755,67.857143,6.998158,12.687813


Saved model to models\st_aug_bapt_2


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,8518.575814,57.142857,6.629834,11.881188


Performance hasn't improved for 1 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,8096.360142,42.682927,6.445672,11.2


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,45.0,7540.292869,83.396947,80.478821,81.911903


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,46.0,7520.755095,82.739212,81.21547,81.97026


Performance hasn't improved for 5 cycles...
Done training after 46 meta cycles.


It's interesting how similar this training performance was to the Matanzas volume above. If this is replicated across more data sets, it probably means something (although I'm not sure what exactly).